In [655]:
import pandas as pd
from pathlib import Path
import datetime

In [656]:
#pd.options.display.float_format = '{:.3f}'.format

In [1]:
data = pd.read_csv('생존분석_ver2.csv')
data=data[data['장애분석']!= 'X']
data.head()

NameError: name 'pd' is not defined

In [658]:
data[data['Lv1']=='통신']['그룹정의'].value_counts()

방범망네트워크    270
스피드게이트     268
전관방송       268
공용회의실      184
얼굴인식       183
울타리감지      167
행정망네트워크    160
CCTV       151
통제식회전문     125
동작감지       104
출입통제        23
비상호출        10
정책홍보         3
Name: 그룹정의, dtype: int64

In [659]:
broken = data[data['status']==1]
cnt = broken['fac_cd'].value_counts()

# 작업을 한번'만' 한 경우
first = list(cnt[cnt==1].index)
first_df = pd.DataFrame({'fac_cd':first})
# 작업을 한번 이상 한 장비 중 첫번째 작업 정보
first_df_2 = data[data['작업횟수']==1]

# 작업을 두번'만' 한 장비
second = list(cnt[cnt==2].index)
second_df = pd.DataFrame({'fac_cd':second})

#작업을 두번 이상 한 장비 중 두번째 작업 정보
second_df_2 = data[data['작업횟수']==2]

# 작업을 세번이상 한 경우
third_df_2 = data[data['작업횟수']>=3]

first_df = pd.merge(first_df,data,on='fac_cd',how='left')
second_df = pd.merge(second_df,data,on='fac_cd',how='left')

predict_cnt_1 = pd.concat([first_df,second_df_2],axis=0)

predict_cnt_2 = pd.concat([second_df,third_df_2],axis=0)
predict_cnt_2 = predict_cnt_2[predict_cnt_2['작업횟수']>=2]
def status_1(df):
    if df['작업횟수'] == 1:
        return 0
    else :
        return 1

def status_2(df):
    if df['작업횟수'] == 2:
        return 0
    else :
        return 1

def input_time(df):
    if df['status'] ==1 :
        return df['작업-작업간격']
    else :
        return df['경과시간']

predict_cnt_1['status'] = predict_cnt_1.apply(status_1,axis=1)
predict_cnt_1['input_time'] = predict_cnt_1.apply(input_time,axis=1)
predict_cnt_2['status'] = predict_cnt_2.apply(status_2,axis=1)
predict_cnt_2['input_time'] = predict_cnt_2.apply(input_time,axis=1)

In [660]:
import matplotlib.pyplot as plt
from lifelines import KaplanMeierFitter

plt.rc('font', family='AppleGothic')
plt.rcParams['axes.unicode_minus'] = False

1. 그룸 정의 별로 학습하기
2. 각 장비별로 인풋값에 넣기
3. 두번 이상 고장난 경우 > status 다르게 해서 봐야함 2번 고장만 난 경우 > 0 3번이상 고장 난 경우 >1

In [661]:

def kplan(그룹정의):
    output_data = pd.DataFrame({
    '대분류' : [],
    '장비분류' :[],
    '동':[],
    '층' :[],
    '장비코드' :[],
    '장비명' : [],
    '장애발생횟수' :[],
    '최근장애일자' :[],
    '7일이내장애확률' :[],
    '30일이내장애확률':[],
    'Lv4':[]
})


    input_data = data[data['그룹정의']==그룹정의]
    # 이때까지 장애가 발생하지 않은 경우 > 처음 발생 하는 확률 찾기
    input_first = input_data[input_data['작업횟수']<=1]
    if len(input_first) >=2:
        kmf_first= KaplanMeierFitter()
        var_first = 0
        input_first.sort_values(by='작업-작업간격', ascending=True)
        kmf_first.fit(durations=input_first['작업-작업간격'], event_observed=input_first['status'])

    # 작업횟수 = 2 인 경우 > 장애가 한번 발생한 장비의 다음 장비 예측하기
    input_second = predict_cnt_1[predict_cnt_1['그룹정의']==그룹정의]
    if len(input_second) >=2:
        kmf_second= KaplanMeierFitter()
        var_second = 0
        input_second.sort_values(by='input_time', ascending=True)
        kmf_second.fit(durations=input_second['input_time'], event_observed=input_second['status'])

    # 작업횟수 >=3 인 경우 > 재발한 장비의 다음 장비 예측하기

    input_third = predict_cnt_2[predict_cnt_2['그룹정의']==그룹정의]
    if len(input_third) >=2:
        kmf_third= KaplanMeierFitter()
        var_third = 0
        input_third.sort_values(by='input_time', ascending=True)
        kmf_third.fit(durations=input_third['input_time'], event_observed=input_third['status'])

    fac_list = list(input_data['fac_cd'].unique())


    if 'var_first' in locals():
        var_first = True
    else :
        var_first = False

    if 'var_second' in locals():
        var_second = True
    else :
        var_second = False

    if 'var_third' in locals():
        var_third = True
    else :
        var_third = False



    for i in fac_list:
        df = input_data[input_data['fac_cd']==i].iloc[-1]
        seven = df['경과시간'] +7
        thirty = seven + 23
        new_data = []
        new_data.append(df['Lv1'])
        new_data.append(df['그룹정의'])
        new_data.append(df['bd_nm'])
        new_data.append(df['fl_cd'])
        new_data.append(df['fac_cd'])
        new_data.append(df['fac_nm'])
        new_data.append(df['작업횟수'])
        new_data.append(df['job_jisi_dt'])
       # 7일 이내 고장 확률
        if df['작업횟수'] == 0 and var_first == True:
            new_data.append(1-kmf_first.predict(seven))

        elif df['작업횟수'] ==1 and var_second == True:
            new_data.append(1-kmf_second.predict(seven))
        elif df['작업횟수'] ==1 and var_first == True:
            new_data.append(1-kmf_first.predict(seven))

        elif df['작업횟수'] >=2 and var_third == True:
            new_data.append(1-kmf_third.predict(seven))
        elif df['작업횟수'] >=2 and var_second == True:
            new_data.append(1-kmf_second.predict(seven))
        elif df['작업횟수'] >=2 and var_first == True:
            new_data.append(1-kmf_first.predict(seven))
        else :
            new_data.append(99999)

        # 30일 이내 고장 확률
        if df['작업횟수'] == 0 and var_first == True:
            new_data.append(1-kmf_first.predict(thirty))

        elif df['작업횟수'] ==1 and var_second == True:
            new_data.append(1-kmf_second.predict(thirty))
        elif df['작업횟수'] ==1 and var_first == True:
            new_data.append(1-kmf_first.predict(thirty))

        elif df['작업횟수'] >=2 and var_third == True:
            new_data.append(1-kmf_third.predict(thirty))
        elif df['작업횟수'] >=2 and var_second == True:
            new_data.append(1-kmf_second.predict(thirty))
        elif df['작업횟수'] >=2 and var_first == True:
            new_data.append(1-kmf_first.predict(thirty))
        else :
            new_data.append(99999)
        new_data.append(df['Lv4'])

        output_data.loc[len(output_data)] = new_data

    return output_data


In [662]:

group = list(data['그룹정의'].unique())
output_data = pd.DataFrame({
    '대분류' : [],
    '장비분류' :[],
    '동':[],
    '층' :[],
    '장비코드' :[],
    '장비명' : [],
    '장애발생횟수' :[],
    '최근장애일자' :[],
    '7일이내장애확률' :[],
    '30일이내장애확률':[],
    'Lv4':[]
})
output_data

,대분류,장비분류,동,층,장비코드,장비명,장애발생횟수,최근장애일자,7일이내장애확률,30일이내장애확률,Lv4


In [663]:
for i in group:
    output = kplan(i)
    output_data = pd.concat([output_data,output],axis=0)
output_data = output_data.drop_duplicates()
output_data['최근장애일자'] = output_data['최근장애일자'].str.replace('00:00:00','')

In [664]:
time_data = pd.read_csv('장비_작업시간.csv')[['jangbi_class_nm_4','job_time']]
#time_data['job_class_nm_4'].dt.days
# 일
time_data['일'] = time_data['job_time'].str[:-13]
time_data=time_data[~time_data['일'].str.contains('-')]
time_data['시간'] = time_data['job_time'].str[-8:]
time_data['시'] = pd.to_datetime(time_data['시간']).dt.hour
time_data['분'] = pd.to_datetime(time_data['시간']).dt.minute
time_data[['일','시','분']] = time_data[['일','시','분']].astype(int)
time_data['총시간']= time_data['일']*24*60 + time_data['시'] * 60 + time_data['분']
time_final = time_data.groupby('jangbi_class_nm_4').median()['총시간']
time_final = time_final.reset_index()
time_final

/var/folders/nd/cl2nzwh13zx7jdly2t5m95340000gn/T/ipykernel_26977/3304006887.py:11: FutureWarning: The default value of numeric_only in DataFrameGroupBy.median is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  time_final = time_data.groupby('jangbi_class_nm_4').median()['총시간']


,jangbi_class_nm_4,총시간
0,CCTV 서버,66.000
1,CCTV 저장장치,83.000
2,CCTV 카메라,88.000
3,DDC 판넬,60.000
4,EPS실,30.500
...,...,...
146,항온항습기 실외기,150.000
147,항온항습기실,60.000
148,헤드,50.000
149,환기조화기,60.000


In [665]:
output_data

,대분류,장비분류,동,층,장비코드,장비명,장애발생횟수,최근장애일자,7일이내장애확률,30일이내장애확률,Lv4
0,기계,개별냉난방기,0,0,200104011076.000,EHP 실내기 #079,0.000,0,0.107,0.118,개별냉난방기 실내기
1,기계,개별냉난방기,0,0,200104011002.000,실내기#117,0.000,0,0.005,0.005,개별냉난방기 실내기
2,기계,개별냉난방기,0,0,200104010987.000,실내기#112,0.000,0,0.005,0.005,개별냉난방기 실내기
3,기계,개별냉난방기,0,0,200104010710.000,실내기#101,0.000,0,0.149,0.161,개별냉난방기 실내기
4,기계,개별냉난방기,0,0,200104010667.000,실내기#57,0.000,0,0.149,0.161,개별냉난방기 실내기
...,...,...,...,...,...,...,...,...,...,...,...
22,전기,옥외 LED 조명,14-3동,P006,300401030060.000,정원등,0.000,0,0.111,0.111,옥외 LED 조명
23,전기,옥외 LED 조명,15-2동,F001,300401030054.000,보안등,0.000,0,0.111,0.111,옥외 LED 조명
24,전기,옥외 LED 조명,15-2동,F001,300401030057.000,수목투사등,0.000,0,0.111,0.111,옥외 LED 조명
25,전기,옥외 LED 조명,15-2동,F001,300401030063.000,태양광 LED 가로등,0.000,0,0.111,0.111,옥외 LED 조명


In [666]:
output_data = pd.merge(output_data,time_final,how='left', left_on='Lv4',right_on='jangbi_class_nm_4')

In [667]:
output_data = output_data.drop(['Lv4','jangbi_class_nm_4'],axis=1)
output_data.rename(columns={'총시간':'작업시간(중앙값)'},inplace=True)
output_data

,대분류,장비분류,동,층,장비코드,장비명,장애발생횟수,최근장애일자,7일이내장애확률,30일이내장애확률,작업시간(중앙값)
0,기계,개별냉난방기,0,0,200104011076.000,EHP 실내기 #079,0.000,0,0.107,0.118,130.000
1,기계,개별냉난방기,0,0,200104011002.000,실내기#117,0.000,0,0.005,0.005,130.000
2,기계,개별냉난방기,0,0,200104010987.000,실내기#112,0.000,0,0.005,0.005,130.000
3,기계,개별냉난방기,0,0,200104010710.000,실내기#101,0.000,0,0.149,0.161,130.000
4,기계,개별냉난방기,0,0,200104010667.000,실내기#57,0.000,0,0.149,0.161,130.000
...,...,...,...,...,...,...,...,...,...,...,...
30777,전기,옥외 LED 조명,14-3동,P006,300401030060.000,정원등,0.000,0,0.111,0.111,55.000
30778,전기,옥외 LED 조명,15-2동,F001,300401030054.000,보안등,0.000,0,0.111,0.111,55.000
30779,전기,옥외 LED 조명,15-2동,F001,300401030057.000,수목투사등,0.000,0,0.111,0.111,55.000
30780,전기,옥외 LED 조명,15-2동,F001,300401030063.000,태양광 LED 가로등,0.000,0,0.111,0.111,55.000


In [668]:
output_data['대분류'].unique()

array(['기계', '통신', '방재', '승강기', '전기'], dtype=object)

In [669]:
output_data

,대분류,장비분류,동,층,장비코드,장비명,장애발생횟수,최근장애일자,7일이내장애확률,30일이내장애확률,작업시간(중앙값)
0,기계,개별냉난방기,0,0,200104011076.000,EHP 실내기 #079,0.000,0,0.107,0.118,130.000
1,기계,개별냉난방기,0,0,200104011002.000,실내기#117,0.000,0,0.005,0.005,130.000
2,기계,개별냉난방기,0,0,200104010987.000,실내기#112,0.000,0,0.005,0.005,130.000
3,기계,개별냉난방기,0,0,200104010710.000,실내기#101,0.000,0,0.149,0.161,130.000
4,기계,개별냉난방기,0,0,200104010667.000,실내기#57,0.000,0,0.149,0.161,130.000
...,...,...,...,...,...,...,...,...,...,...,...
30777,전기,옥외 LED 조명,14-3동,P006,300401030060.000,정원등,0.000,0,0.111,0.111,55.000
30778,전기,옥외 LED 조명,15-2동,F001,300401030054.000,보안등,0.000,0,0.111,0.111,55.000
30779,전기,옥외 LED 조명,15-2동,F001,300401030057.000,수목투사등,0.000,0,0.111,0.111,55.000
30780,전기,옥외 LED 조명,15-2동,F001,300401030063.000,태양광 LED 가로등,0.000,0,0.111,0.111,55.000


In [670]:
arr = list(output_data['대분류'].unique())
arr

['기계', '통신', '방재', '승강기', '전기']

In [671]:
writer=pd.ExcelWriter('최종결과_ver2.xlsx', engine='openpyxl')

for i in arr:

    output_data_final = output_data[output_data['대분류']==i]
    output_data_final.to_excel(writer, sheet_name = f'{i}_장비별')

    output_data_final_broken = output_data_final.groupby(['장비분류','동']).sum()['장애발생횟수']

    output_data_final_prob_max = output_data_final.groupby(['장비분류','동']).max()[['7일이내장애확률','30일이내장애확률']]
    output_data_final_prob_max.columns = ['7일이내장애확률(최대)','30일이내장애확률(최대)']

    output_data_final_prob_min = output_data_final.groupby(['장비분류','동']).min()[['7일이내장애확률','30일이내장애확률']]
    output_data_final_prob_min.columns = ['7일이내장애확률(최소)','30일이내장애확률(최소)']

    output_data_final_prob_mean = output_data_final.groupby(['장비분류','동']).mean()[['7일이내장애확률','30일이내장애확률']]
    output_data_final_prob_mean.columns = ['7일이내장애확률(평균)','30일이내장애확률(평균)']

    output_data_final_prob_median = output_data_final.groupby(['장비분류','동']).median()[['7일이내장애확률','30일이내장애확률']]
    output_data_final_prob_median.columns = ['7일이내장애확률(중앙)','30일이내장애확률(중앙)']

    output_data_final_group = pd.concat([output_data_final_broken,output_data_final_prob_max,output_data_final_prob_min,output_data_final_prob_mean,output_data_final_prob_median],axis = 1)
    output_data_final_group.to_excel(writer, sheet_name = f'{i}_동별')

writer.save()


/var/folders/nd/cl2nzwh13zx7jdly2t5m95340000gn/T/ipykernel_26977/2373678955.py:8: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  output_data_final_broken = output_data_final.groupby(['장비분류','동']).sum()['장애발생횟수']
/var/folders/nd/cl2nzwh13zx7jdly2t5m95340000gn/T/ipykernel_26977/2373678955.py:16: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  output_data_final_prob_mean = output_data_final.groupby(['장비분류','동']).mean()[['7일이내장애확률','30일이내장애확률']]
/var/folders/nd/cl2nzwh13zx7jdly2t5m95340000gn/T/ipykernel_26977/2373678955.py:19: FutureWarning: The default value of numeric_only in DataFrameGroupBy.median is deprecated